In [2]:
import transformers
import pandas as pd
import os
import re
from unidecode import unidecode
from tqdm import tqdm
pd.options.display.max_colwidth = 150

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("yiyanghkust/finbert-esg-9-categories")
model = AutoModelForSequenceClassification.from_pretrained("yiyanghkust/finbert-esg-9-categories")

Downloading: 100%|██████████| 1.17k/1.17k [00:00<00:00, 1.17MB/s]
Downloading: 100%|██████████| 226k/226k [00:00<00:00, 300kB/s]  
Downloading: 100%|██████████| 439M/439M [00:28<00:00, 15.5MB/s] 


In [4]:
import nltk

# Download the Punkt tokenizer
nltk.download('punkt')


def get_sentences_from_file(filename):
    # Read the text file
    with open(filename, 'r',encoding='utf-8') as f:
        text = f.read()

    # Split the text into sentences
    sentences = nltk.sent_tokenize(text)

    return(sentences)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pivsa\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [5]:
reSentenceSplitter = re.compile(
    "(?i:\\bYum!\\s*(?=Brands?\\b))|" +
    "(?i:\\b(?:inc|corp|ltd)\\.\\s*\\()|" +
    "(?i:\\b(?:mrs|messrs|sen|esq|adv|prof|rev|gov|gen|rep|hon|adj|oblig|tbk|dev|inv|invs|opn|constr|conces|(?-i:Med|Cap|Develop|Met))\\.\\s*(?=[A-Z'\"\\(]))|" +
    "\\b(?i:[a-z][a-z][a-z]+(?:'t)?|" +
    "(?-i:we|us|in|to|at|of|on|as|be|by|do|go|he|is|it|me|so|up|no|US|UK|EU|MW)|" +
    "'s|" +
    "[A-Z]&(?:amp;)?[A-Z]+|" +
    "\\s*\\([A-Z]+\\)|" +
    "\\d+[%$sx]?|" +
    "\\)(?=\\.)|" +
    "[A-Z]+\\d+)" +
    "[\"'\\)]*[!\\.\\?]+[\"'\\)]*( +)" +
    "(?=" + "(?:<[^\\>][^>]*>)*" +
    "(?:[\"']+(?:<[^\\>][^>]*>)*)?" +
    "\\(?" +
    "(?:[A-Z][a-z]|Q[1-4]|\\d+[A-Z]+ |[AI] |I'|U\\.S\\b|[A-Z]+\\b|(?:19|20)\\d\\d ))") 

def NormalizeString(s, doUnescape=False):
    if doUnescape: s = html.unescape(s)
    # s = unicodedata.normalize('NFKD', s).encode('ascii', 'ignore').decode()
    s = unidecode(s)
    s = re.sub(r'[<{]', '[', s)
    s = re.sub(r'[>}]', ']', s)
    s = re.sub(r' ([\.,\?!]) ', r'\1 ', s)
    s = s.replace('(', '')
    s = s.replace(')', '')
    s = s.replace(' .', '.')
    return re.sub(r'\s+', ' ', s).strip()

def GetSentences(text):
    # print(text)
    text = re.sub(r'\[\[.*?\]\]', '', text)
    text = NormalizeString(text)
    sentences = []
    for paragraph in text.split('\n'):
        paragraph = paragraph.strip()
        if not paragraph: continue
        iAt = 0
        for M in reSentenceSplitter.finditer(paragraph):
            if not M.group(1): continue
            sent = paragraph[iAt: M.start(1)].strip()
            # print(iAt, "'" + M.group(1) + "'", M.start(1), M.end(1), sent)
            # print()
            iAt = M.end(1)
            if not sent: continue
            sentences.append(sent)
        sent = paragraph[iAt:].strip()
        if not sent: continue
        sentences.append(sent)
    return sentences


def split_into_sentences(text):
    if not isinstance(text, str):
        return []
    if len(text.split(" ")) < 2:
        return []
    sentences = GetSentences(text)
    sentences = [x for x in sentences if len(x.split(" ")) > 3]
    # if len(sentences) < 3:
    #     print(sentences)
    return sentences


In [7]:
# Create an empty DataFrame
df = pd.DataFrame(columns=["year", "sentence"])

# Iterate over all files in the "Conoco" directory
for file_name in os.listdir("AMCOR"):
    if file_name.endswith(".txt"):
        # Extract the year from the file name
        year = file_name.replace(".txt", "")

        # Read the file and split it into sentences
        with open(os.path.join("AMCOR", file_name),encoding='utf-8') as f:
            content = f.read()
            #sentences = re.split(r'(?<=[^A-Z].[.?]) +(?=([^A-Z]|$))|\n', content)
            sentences = split_into_sentences(content)
            # Iterate over the sentences and add them to the DataFrame
            for sentence in sentences:
                new_row = pd.DataFrame({"year": [year], "sentence": [sentence]})
                df = pd.concat([df, new_row], ignore_index=True)

In [4]:
sentences = get_sentences_from_file('C:/Users/ronen/Dropbox/NLP/Code/ALCOA CORP_2021.txt')

In [5]:
len(sentences)

2128

In [14]:
results = []

id2label =  {
    0: "Climate Change",
    1: "Natural Capital",
    2: "Pollution & Waste",
    3: "Human Capital",
    4: "Product Liability",
    5: "Community Relations",
    6: "Corporate Governance",
    7: "Business Ethics & Values",
    8: "Non-ESG"
  }

for sentence in tqdm(df['sentence']):
    try:
        input_ids = tokenizer.encode(sentence, return_tensors='pt')
        output = model(input_ids)

        # Determine the most likely class
        # Determine the index of the class with the highest probability
        prediction = output.logits.argmax().item()

        # Map the index to a class label
        category = id2label[prediction]
        # Store the result
        results.append({
            'sentence': sentence,
            'esg_category': category
        })
    except Exception as e:
        print(f"An error occurred for the sentence: {sentence}")
        print(e)



 85%|████████████████████████████████████████████████████████████████████████▍            | 5141/6032 [05:29<00:52, 17.05it/s]

An error occurred for the sentence: Maintain net-zero future for Boeing operations Scope 1 and 2 through conservation and renewable energy Partner with supply chain for responsible business practices Top quartile Recordable Injury Rate among sector benchmarked performance 90% believe their manager supports their well-being Drive aerospace safety via global aerospace safety initiatives to maintain downward trend of worldwide commercial jet fleet 10 Year Moving Average Fatal Accident Rate Increase representation of women globally and underrepresented racial/ ethnic minorities in the U.S. Achieve 55% absolute reduction in Scope 1 and 2 GHG from 2017 baseline1 Maintain net-zero emissions for Scope 1 and 2 Achieve 100% renewable electricity Work with our suppliers to increase GHG reporting and proactively address climate change driven risks Support the commercial aviation industry's ambition to achieve net-zero carbon emissions for global civil aviation operations by 2050 Current and future

100%|█████████████████████████████████████████████████████████████████████████████████████| 6032/6032 [06:30<00:00, 15.45it/s]


In [17]:
sent_df = pd.DataFrame(results)

In [19]:
df['esg_category'] = sent_df['esg_category']

In [20]:
df.style.set_properties(subset=['sentence'], **{'width': '900px'}).set_table_styles([{'selector': 'td', 'props': [('white-space', 'normal'), ('text-align', 'left')]}])

,year,sentence,esg_category
0,2013,"At Boeing, we aspire to be the strongest, best and best-integrated aerospace-based company in the world-for today and tomorrow.",Non-ESG
1,2013,"Boeing is the world's largest aerospace company and leading manufacturer of commercial airplanes and defense, space and security systems.",Non-ESG
2,2013,"The top U.S. exporter, Boeing supports airlines and U.S. and allied government customers in more than 150 countries.",Community Relations
3,2013,"Our products and tailored services include commercial and military aircraft, satellites, weapons, electronic and defense systems, launch systems, advanced information and communication systems, and performance-based logistics and training.",Non-ESG
4,2013,"With corporate offices in Chicago, Boeing employs more than 174,000 people across the United States and in 70 countries.",Human Capital
5,2013,"In addition, our enterprise leverages the talents of hundreds of thousands of skilled people working for Boeing suppliers worldwide.",Human Capital
6,2013,Cover photo: The innovative Phantom Eye is powered by clean-burning hydrogen and emits only water in the atmosphere.,Climate Change
7,2013,"Boeing photo Photo above: The 737 MAX, currently in development, features a 13 percent smaller carbon footprint than today's most fuel-efficient single-aisle airplanes.",Climate Change
8,2013,"Five years ago, we set ambitious goals to reduce our environmental footprint while significantly growing our business.",Climate Change
9,2013,"Thanks to the dedication and hard work of everyone at Boeing, we met those goals, and are prepared to make further progress in the years ahead.",Human Capital


In [21]:
df['esg_category'].value_counts()

Climate Change              2077
Human Capital                743
Pollution & Waste            714
Community Relations          584
Natural Capital              531
Product Liability            492
Non-ESG                      439
Corporate Governance         247
Business Ethics & Values     204
Name: esg_category, dtype: int64

In [23]:
df.groupby("year")["esg_category"].value_counts()

year  esg_category            
2013  Climate Change              201
      Pollution & Waste            83
      Natural Capital              41
      Non-ESG                      34
      Human Capital                32
                                 ... 
2022  Product Liability            82
      Business Ethics & Values     81
      Non-ESG                      67
      Pollution & Waste            52
      Natural Capital              39
Name: esg_category, Length: 90, dtype: int64

In [26]:
df.query("esg_category == 'Climate Change'").groupby("year").size().div(df.groupby("year").size()).mul(100)

year
2013    44.567627
2014    41.379310
2015    43.218085
2016    33.666667
2017    42.200000
2018    44.266055
2019    47.703180
2020    40.331492
2021    18.211921
2022    20.172084
dtype: float64

In [27]:
df.groupby(["year", "esg_category"]).size().div(df.groupby("year").size()).mul(100).groupby(level=0).idxmax()

year
2013    (2013, Climate Change)
2014    (2014, Climate Change)
2015    (2015, Climate Change)
2016    (2016, Climate Change)
2017    (2017, Climate Change)
2018    (2018, Climate Change)
2019    (2019, Climate Change)
2020    (2020, Climate Change)
2021     (2021, Human Capital)
2022     (2022, Human Capital)
dtype: object

In [28]:
most_frequent = df.groupby(["year", "esg_category"]).size().div(df.groupby("year").size()).mul(100)
most_frequent_category = most_frequent.groupby(level=0).idxmax().apply(lambda x: x[1])
most_frequent_percentage = most_frequent.groupby(level=0).max()
result = pd.concat([most_frequent_percentage, most_frequent_category], axis=1)
result.columns = ['percentage', 'category']

In [29]:
result

,percentage,category
year,,
2013,44.567627,Climate Change
2014,41.379310,Climate Change
2015,43.218085,Climate Change
2016,33.666667,Climate Change
2017,42.200000,Climate Change
2018,44.266055,Climate Change
2019,47.703180,Climate Change
2020,40.331492,Climate Change
2021,22.516556,Human Capital


In [30]:
most_frequent = df.groupby(["year", "esg_category"]).size().div(df.groupby("year").size()).mul(100)
most_frequent_category = most_frequent.groupby(level=0).idxmin().apply(lambda x: x[1])
most_frequent_percentage = most_frequent.groupby(level=0).min()
result = pd.concat([most_frequent_percentage, most_frequent_category], axis=1)
result.columns = ['percentage', 'category']

In [31]:
result

,percentage,category
year,,
2013,1.330377,Business Ethics & Values
2014,0.718391,Business Ethics & Values
2015,0.664894,Business Ethics & Values
2016,1.166667,Business Ethics & Values
2017,1.800000,Corporate Governance
2018,1.376147,Business Ethics & Values
2019,0.706714,Business Ethics & Values
2020,1.657459,Business Ethics & Values
2021,3.752759,Natural Capital
